In [48]:
import mysql.connector

conn = mysql.connector.connect(username= "root",
                               host = "127.0.0.1",
                               password = "Ananthu@123",
                               database = "data_spark")

cursor = conn.cursor()
import pandas as pd

In [49]:
df_stores = pd.read_csv('Stores.csv')

print(df_stores.head())

print(df_stores.info())

df_stores.columns = df_stores.columns.str.lower()
df_stores.columns = df_stores.columns.str.replace(" ","_")

df_stores['open_date'] = pd.to_datetime(df_stores['open_date'])

df_stores['square_meters'] = df_stores['square_meters'].fillna(0.0)

   StoreKey    Country                         State  Square Meters  Open Date
0         1  Australia  Australian Capital Territory          595.0   1/1/2008
1         2  Australia            Northern Territory          665.0  1/12/2008
2         3  Australia               South Australia         2000.0   1/7/2012
3         4  Australia                      Tasmania         2000.0   1/1/2010
4         5  Australia                      Victoria         2000.0  12/9/2015
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   StoreKey       67 non-null     int64  
 1   Country        67 non-null     object 
 2   State          67 non-null     object 
 3   Square Meters  66 non-null     float64
 4   Open Date      67 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.7+ KB
None


In [50]:
create_store_table = """CREATE TABLE stores (
    storekey INT PRIMARY KEY,
    country VARCHAR(100),
    state VARCHAR(100),
    square_meters DECIMAL(10, 2),
    open_date DATE
);"""
cursor.execute(create_store_table)
conn.commit()

In [51]:
import pymysql  # or your preferred MySQL library
import pandas as pd

# Sample DataFrame
# df_stores = pd.DataFrame({
#     'storekey': [...],
#     'country': [...],
#     'state': [...],
#     'square_meters': [...],
#     'open_date': [...]
# })

# Establish connection to the database
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',           # Corrected from 'username' to 'user'
    password='Ananthu@123',
    database='data_spark'
)
cursor = conn.cursor()

insert_stores_values = '''
    INSERT INTO stores (storekey, country, state, square_meters, open_date)
    VALUES (%s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
    country=VALUES(country),
    state=VALUES(state),
    square_meters=VALUES(square_meters),
    open_date=VALUES(open_date)
'''

# Iterate over DataFrame rows and execute the insert statement
for index, row in df_stores.iterrows():
    cursor.execute(insert_stores_values, (
        row['storekey'],
        row['country'],
        row['state'],
        row['square_meters'],
        row['open_date']
    ))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()
